# Bipartites Matching
Bipartite Graphen lassen sich in zwei Teilmengen unterteilen, sodass Kanten nur zwischen diesen, nicht aber innerhalb dieser Teilmengen verlaufen. Mit ihnen lassen sich spezielle Zuordnungsprobleme modellieren, die in diesem Abschnitt betrachtet werden sollen.

In [ ]:
import networkx as nx

from tui_dsmt.graph import Hall, BipartiteFlow
from tui_dsmt.graph.datasets import draw_bi
from tui_dsmt.graph.datasets import load_bi_flow, load_bi_small, load_bi_recommendations

small_graph = load_bi_small()
flow_graph = load_bi_flow()

## Inhaltsverzeichnis
- [Beispiel: Praktika](#Beispiel-Praktika)
- [Definition](#Definition)
- [Naiver Ansatz](#Naiver-Ansatz)
- [Satz von Hall](#Satz-von-Hall)
- [Maximum Flow Algorithmen in bipartiten Graphen](#Maximum-Flow-Algorithmen-in-bipartiten-Graphen)
- [Beispiel: Empfehlungssysteme](#Beispiel-Empfehlungssysteme)

## Beispiel: Praktika
Studenten müssen ein Praktikum in einem der vielen Praktikumsbetriebe absolvieren. Allerdings schränkt jeder Praktikumsbetrieb sein Angebot auf bestimmte Fachrichtungen und Vorkenntnisse ein. Es kommt also nicht jeder Student für jeden Praktikumsplatz in Frage.

Modelliert man jeweils Studenten (Buchstaben) und Betriebe (Ziffern) als Knoten und die Möglichkeit eines Praktikums als Kante, ergibt sich das bereits bekannte Bild:

In [ ]:
draw_bi(small_graph)

Um ein gutes Matching zu gewährleisten, müssen nun aber einige Dinge beachtet werden:
- $A$ und $D$ können jeweils nur einem Betrieb zugeordnet werden. Wird eine andere Person den besagten Betrieben zugeordnet, so gehen $A$ oder $D$ leer aus.
- Für Betrieb $3$ kommt nur Student $C$ in Frage. Wird dieser nicht Betrieb $3$ zugeordnet, geht Betrieb $3$ leer aus.

## Definition
Ein Graph $G = (V, E)$ ist genau dann bipartit, wenn sich seine Knoten $V$ in zwei disjunkte Teilmengen $A$ und $B$ aufteilen lassen, sodass keine Kanten innerhalb der Mengen verlaufen. Es gilt also $A \cup B = V$ und $A \cap B = \emptyset$ und $\forall (a, b) \in E : a \in A \land b \in B \lor a \in B \land b \in A$.

Eine Menge $M \subseteq E$ heißt Matching, wenn jeder Knoten im Graph $(V, M)$ maximal Knotengrad $1$ besitzt bzw. keine zwei Kanten aus $M$ einen Knoten gemeinsam haben. (Jeder Knoten wird also maximal einem anderen Knoten zugeordnet.) Je nach Modellierung des Problems lassen sich Matchings in verschiedene Kategorien einteilen:
1. Ein Matching heißt **größtmöglich**, falls es die größte Kantenanzahl unter allen Matchings des Graphen besitzt.
2. Ein Matching heißt **perfekt**, falls jeder Knoten gematcht wurde bzw. halb so viele Kanten in $M$ enthalten sind wie $V$ Knoten besitzt.

## Naiver Ansatz
Brute-Force funktioniert (fast) immer. Ein erster Ansatz könnte also sein, alle möglichen Kombinationen von Kanten zu testen und die vielversprechendsten Kandidaten zu übernehmen. Vorgegangen wird nach folgendem Schema:
- Jede verfügbare Kante aus $E$ kann entweder in $M$ übernommen oder nicht übernommen werden. Es ergeben sich daher $2^{|E|}$ potentielle Kandidaten.
- Sobald ein Knoten innerhalb eines Kandidaten den Grad $2$ erreicht, ist das Matching ungültig.
- Die verbleibenden Kandidaten für $M$ werden bewertet und der beste ausgewählt.

In [ ]:
# Diese Funktion bewertet ein Matching.
def rate_matching(graph):
    degree_sum = 0

    for node, degree in graph.degree:
        # Matchings mit Knotengrad > 1 ausschließen
        if degree > 1:
            return None

        # Matchings mit Punktzahl versehen
        degree_sum += degree

    return degree_sum


# Diese Funktion generiert alle 2^|E| Kandidaten für das Matching.
def enumerate_matchings(graph, edges):
    # keine Kanten mehr vorhanden
    # Rekursionsabbruch
    if len(edges) == 0:
        # Matching bewerten
        rating = rate_matching(graph)

        # falls Matching gültig ist
        if rating is not None:
            yield graph.copy(), rating

        return

    # eine Kante entnehmen
    edge = edges[0]

    # rekursiver Aufruf mit dieser Kante
    graph.add_edge(*edge)
    yield from enumerate_matchings(graph, edges[1:])

    # rekursiver Aufruf ohne diese Kante
    graph.remove_edge(*edge)
    yield from enumerate_matchings(graph, edges[1:])


# Anlegen des leeren Graphen und der Kantenliste
empty_matching = nx.Graph()
small_graph_edges = list(small_graph.edges)

# alle Matchings generieren
matchings = list(enumerate_matchings(empty_matching, small_graph_edges))

# beste Matchings suchen
_, max_rating = max(matchings, key=lambda x: x[1])
best_matchings = [graph for graph, rating in matchings if rating == max_rating]

for graph in best_matchings:
    draw_bi(graph)

Für nicht-triviale Graphen ergibt sich aber das Problem der Komplexität. Bei zehn möglichen Kanten müssen $1024$ Kandidaten geprüft werden. Bei zwanzig möglichen Kanten sind es bereits mehr als eine Million Kandidaten. Auch wenn es durch Umbau des Algorithmus möglich ist, viele potentielle Matchings bereits sehr früh auszuschließen, ist eine erschöpfende Suche in größeren Graphen keine realistische Option.

## Satz von Hall
Der Satz von Hall ist ein notwendiges und (nicht offensichtlich) sogar hinreichendes Kriterium zur Existenz eines Matchings. Der Satz ist auch als Heiratssatz bekannt, da das zugrundeliegende Problem häufig mit Damen, die verheiratet werden sollten, aber nicht mit jedem zur Verfügung stehenden Kandidaten zufrieden sind, veranschaulicht wurde.

Sei der Graph nun wieder gegeben als $G = (V, E)$ und bipartit, es gibt also disjunkte Teilmengen $A$ und $B$ der Knotenmenge $V$. Weiterhin existiert eine Abbildung $N$, die jeder Menge von Knoten die ihnen durch eine Kante verbundene Menge von Knoten bzw. Nachbarn zuordnet:

$$
N(A) = \{ b \in B : \exists (a, b) \in E \land a \in A \}
$$


Nach dem Satz von Hall sind nun die folgenden Aussagen äquivalent:
1. Es gibt ein Matching, in dem jedem Knoten von $A$ ein Knoten aus $B$ zugeordnet wird.
2. Für jede Teilmenge $S \subseteq A$ gilt $|S| \leq |N(S)|$.

Auch zur Prüfung dieses Kriteriums ist die Berechnung aller Teilmengen von $A$ und $B$ notwendig, sodass insgesamt $2^{|A|}+2^{|B|}$ Teilmengen betrachtet werden müssen. Anhand der nachfolgenden Animation können Sie den Graphen mit Hilfe des Satzes von Hall prüfen.

In [ ]:
A = {'A', 'B', 'C', 'D'}
B = {'1', '2', '3', '4'}

Hall(small_graph, A, B)

Bereits kleine Änderungen können dafür sorgen, dass die Bedingung für den Satz von Hall verletzt wird und somit kein perfektes Matching mehr möglich ist.

In [ ]:
another_G = small_graph.copy()
another_G.add_edge('E', '4')

Hall(another_G, {*A, 'E'}, B)

## Maximum Flow Algorithmen in bipartiten Graphen
Eine der einfachsten Methoden zur Lösung des Matching-Problems ist die Anwendung von Maximum-Flow-Algorithmen. Dazu werden die künstlichen Knoten für die Quelle $s$ und die Senke $t$ eingefügt. $s$ erhält jeweils eine Kante zu allen Knoten der Teilmenge $A$, während $t$ analog eine Kante zu jedem Knoten der Teilmenge $B$ erhält. Als Kapazität wird für jede einzelne Kante zunächst $1$ angenommen.

In [ ]:
BipartiteFlow(flow_graph)

Durch Anwendung des aus dem vorangegangenen Abschnitt bekannten Algorithmus von Ford und Fulkerson entsteht ein größtmögliches Matching in geringerer Laufzeit.

**Verständnisfrage:** Wie müssen die Kapazitäten verändert werden, wenn beispielsweise ein Betrieb mehrere Studenten aufnehmen kann.

## Beispiel: Empfehlungssysteme
In diesem Szenario stehen eine Menge von Kunden $\{A, B, C, D\}$ einer Menge von Werbebuchungen $\{1, 2, 3\}$ gegenüber. Jedem Nutzer kann jede Werbung nur einmal ausgespielt und maximal zwei Werbeeinblendungen angezeigt werden. Jede Werbebuchung ist in der Anzahl ihrer Ausspielungen durch das Budget des Auftraggebers limitiert.

Die vorangegangenen Informationen lassen sich innerhalb eines Graphen modellieren und mit einem der oben gezeigten Algorithmen für ein Zuordnungsproblem verwenden.

In [ ]:
r_graph = load_bi_recommendations()
BipartiteFlow(r_graph)

Auf Basis vorangegangener Käufe oder Bewertungen kann geschätzt werden, wie geeignet eine bestimmte Werbung für einen Kunden ist. Wird diese Information als Gewicht einer Kante kodiert und der Algorithmus so modifiziert, dass nach einer Zuordnung mit einer maximalen Summe von Kantengewichten gesucht wird, sind die Werbeeinblendungen personalisiert.